<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/139_%E9%96%8B%E7%99%BA%E7%92%B0%E5%A2%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

開発環境
========

WSL2
----

### 仮想化と WSL2

現代のアプリケーション開発では、複数のプログラミング言語を同時に使ったり、データベースサーバーといった専門的なソフトウェアと連携するのが一般的である。これらはプロジェクトごとに異なる種類やバージョンが必要になることも少なくない。その結果、個人の PC（ローカル環境）に複数のプロジェクトの開発環境を共存させることが難しくなるという課題が生まれる。

この課題を解決するため、ローカル環境を汚さず、アプリケーション実行環境をプロジェクトごとに独立させる**仮想化**（virtualization）の技術が用いられる。仮想化は、物理的なハードウェアを抽象化し、論理的に分離された複数の環境を同時に動作させる仕組みであり、開発者は一台の PC 上に複数の独立した実行環境を用意し、それぞれを隔離して利用できる。Python の仮想環境だけでは Python ライブラリしか隔離できないため、アプリケーション実行環境をまるごと仮想化する方法がとられる。

その有力な選択肢の一つが、Windows 上で Linux 環境を動作させる **WSL2**（Windows Subsystem for Linux version 2）である。WSL2 は、「仮想マシン」（virtual machine; VM）と呼ばれる、コンピューターの動作をエミュレートする仮想化技術を用いて、この独立した環境を構築する。

一般的に、仮想マシンは**ハイパーバイザー**（hypervisor）という土台の上で OS（ゲストOS）をまるごと動かすため、動作が重くなる傾向がある。しかし、WSL2 が利用する仮想マシンは、Microsoft 製の Hyper-V をベースに、最小限の Linux カーネルのみを動かすように最適化されているため、非常に軽量に動作するという利点がある。

WSL2 では、Linux のファイルは「ext4」という形式でフォーマットされた仮想ディスク（.vhdx ファイル）に保存され、Windows 環境から隔離される。一方、Windows エクスプローラーは WSL2 と連携し、仮想ディスク内のファイルにアクセス可能である（左ペインの Linux アイコンからたどることができる）。

WSL2 の動作には、CPU の仮想化支援機能が必要である。デフォルトでこの機能が有効になっていることがある。これは「タスク マネージャー」を開いて「パフォーマンス」→「CPU」の中で「仮想化: 有効」となっていることで確認できる。そうでない場合は、UEFI（BIOS）の設定から「VT-x」または「Intel Virtualization Technology」を有効にする必要がある。

WSL2 機能を有効化して Ubuntu をインストールする手順は、以下の動画の中のチャプター

  * 4.Windows Subsystem for Linux
  * 5.Ubuntu

が詳しい。

In [ ]:
from IPython.display import HTML
HTML('''<iframe width="640" height="360" src="https://www.youtube.com/embed/2l_nSudnKs4?start=575"
  frameborder="0" allow="accelerometer; clipboard-write; encrypted-media; gyroscope; picture-in-picture"
  referrerpolicy="strict-origin-when-cross-origin"></iframe>''')

WSL は https://github.com/microsoft/WSL 上でオープンに開発されており、更新頻度が高い。WSL をアップデートするには、`wsl --update` を実行する。

``` shell
PS> wsl --update
更新プログラムを確認しています。
Linux 用 Windows サブシステムをバージョン 2.6.1 に更新しています。
```

### ディストリビューションの実行と自動マウント

``` shell
wsl ~
```

`~` を `wsl` と一緒に使用して、既定のディストリビューションの実行をユーザーのホームディレクトリから開始できる。

実行中のディストリビューション側では、自動的に Windows ファイルシステムのドライブが

``` shell
/mnt/<ドライブ名>
```

にマウントされる。

例えば、`/mnt/c` に Windows ファイルシステムの C ドライブがマウントされており、`/mnt/c/Windows/System32/drivers/etc/hosts` は Windows ファイルシステムの `C:\Windows\System32\drivers\etc\hosts` にアクセスする。

`~` を付けずに `wsl` を単独で実行する場合、Windows の現在の作業ディレクトリにマウントされたディレクトリを作業ディレクトリとしてディストリビューションを実行する。

Linux のファイルシステムとは根本的に異なる Windows のファイルシステムにアクセスするため、自動マウントは非常にパフォーマンスが悪いことに注意する。

ディストリビューションの実行については、以下のオプションを指定できる。

| オプション | 説明 |
|:---|:---|
| `--distribution, -d <DistroName>` | 指定されたディストリビューションを実行する |
| `--user, -u <UserName>` | 指定されたユーザーとして実行する |
| `--cd <Directory>` | 指定されたディレクトリを現在の作業ディレクトリとして設定する。`/` 文字で始まる場合、絶対 Linux パスとして解釈され<br /><br />る。それ以外の場合、値は絶対 Windows パスである必要がある |

ディストリビューションは一定時間使用されないと自動的に停止する。

``` shell
wsl --shutdown
```

実行中の全てのディストリビューションと WSL2 の仮想マシンを直ちに終了する。

### ディストリビューションの管理

■ ディストリビューションの状態の一覧表示

``` shell
wsl --list --verbose
```

インストールされているディストリビューションの状態を一覧表示する。このコマンドは、`wsl -l -v` と入力することもできる。

``` shell
PS> wsl -l -v
  NAME      STATE           VERSION
* Ubuntu    Stopped         2
```

`*` のマークは、既定のディストリビューションを示す。既定のディストリビューションは、`wsl` コマンドがデフォルトで実行するディストリビューションである。`NAME` には、ディストリビューションの名前が表示される。`STATE` には、実行中なら `Running`、停止中なら `Stopped` が表示される。`VERSION` には、WSL のバージョンが表示されるが、WSL2 ではすべて `2` である。

■ 既定のディストリビューションの設定

``` shell
wsl --set-default <Distribution Name>
```

既定のディストリビューションを変更する。名前でディストリビューションを指定する。このコマンドは、`wsl -s` と入力することもできる。

■ ディストリビューションのエクスポート

``` shell
wsl --export <Distro> <FileName>
```

指定したディストリビューションの現在の状態をスナップショットとして tar ファイルに保存する。ディストリビューションの名前と、tar ファイルのパスを指定する。

``` shell
PS> wsl --export Ubuntu ./Ubuntu_20250712.tar
エクスポートが進行中です。これには数分かかる場合があります。 (2320 MB)

この操作を正しく終了しました。
```

上の例では、Ubuntu の現在の状態をカレントディレクトリの `Ubuntu_20250712.tar` に保存している。

■ ディストリビューションのインポート

``` shell
wsl --import <Distro> <InstallLocation> <FileName>
```

指定した tar ファイルから新しいディストリビューションをインストールして登録する。登録する名前、インストール先のパス、tar ファイルのパスを指定する。

``` shell
PS> wsl --import ubuntu001 "$Env:LOCALAPPDATA\wsl\ubuntu001" ./Ubuntu_20250712.tar
この操作を正しく終了しました。
PS> wsl -l -v
  NAME         STATE           VERSION
* Ubuntu       Stopped         2
  ubuntu001    Stopped         2
```

上の例では、`ubuntu001` という名前で登録するディストリビューションを、カレントディレクトリの `Ubuntu_20250712.tar` から `$Env:LOCALAPPDATA\wsl\ubuntu001` ディレクトリにインストールする。

インストール先のディレクトリ配下に、以下のファイルが作成される。

  * .vhdx ファイル（tar ファイルの中身がインポートされる）
  * .ico ファイル（アイコンファイル）

インポートされたディストリビューションは、`wsl --distribution` コマンドで実行できる。自動的には既定のディストリビューションにならないので注意する。

また、ターミナルを再起動すると、ターミナルのプロファイルが自動的に追加されていることを確認できる。

【制限事項】  
追加されたプロファイルのアイコンパスは正しく設定されない。ターミナルの設定を開き、プロファイルのアイコンパスを修正する必要がある。

■ ディストリビューションのアンインストール

``` shell
wsl --unregister <Distro>
```

ディストリビューションをアンインストールする。名前でディストリビューションを指定する。

``` shell
PS> wsl --unregister ubuntu001
登録解除。
この操作を正しく終了しました。
```

上の例では、`ubuntu001` がアンインストールされ、インストール時に作成されたディレクトリ（`$Env:LOCALAPPDATA\wsl\ubuntu001`）が中身ごと削除される。

既定のディストリビューションをアンインストールした場合、WSL2 は自動的に、残っているディストリビューションの中から新しい既定のディストリビューションを割り当てる。

【制限事項】  
ターミナルのプロファイルは、メニュー上非表示となるが、設定ファイルから自動的に削除されないという問題がある。ターミナルの最新バージョンでは、設定タブを開くと、非表示となったプロファイルが下図のようにアイコンで示されるようになったが、手動で削除して設定を保存する必要がある。

<img src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAT4AAAAiCAYAAADGW7VkAAAAAXNSR0IB2cksfwAAAARnQU1BAACx
jwv8YQUAAAAgY0hSTQAAeiYAAICEAAD6AAAAgOgAAHUwAADqYAAAOpgAABdwnLpRPAAABtpJREFU
eNrt3H9M1Pcdx/EnHDTajfJTZjhb7jvbgXhb2aphnZZNg3akR5lLL8sMY2ThsnWUMTE5ZtIE76Jt
xyL+YbSDMx2zV1N2rVUwoYnMW5xkGgTdKCci9Htd+TrCgmyVShXq9ofhW653yimnPfT9+Iv7fj7f
7334fvm+eH8+HN8Yk8n0P4QQ4j4SK6dACCHBJ4QQEnxCCCHBJ4QQEnxCCCHBJ4QQEnxCCCHBJ4QQ
X5i4uex8csXc3jzvlFwAIe6bsImP55FHs1i6zEzOijyMpqX0/f00bzc18L1nfsCqgu8DoPnfx9d1
kkFfD/88f46pqcmIjyVmLv+5IcEnhAhH5mPZPF/7CrGxgZPMSuszVLy4jd3bXuSlvW+QkJgU0D41
eZU9ji0MvX8+ouMxJCUlbb3dnW0Z4fWLz3qCa6P/Ctq+94L8QAhxryv6STnP2V4gJiYmYHtH+7sY
4gys/+GPmLx6FbW/j+xvfDOgT6zBQN7apzHExTHQ+4+IjSnia3wPvVBPukcl3aMCkO5RSd72lv5a
CHF/VXpPFT4bsu3Nhl0UbSwDYG3RBtoPvcWl//4nZN+1xVaMytLoDL50j8qC725g/PWXGLEqUXHi
vV4vZWVlUfXDYLFY8Hq9qKpKd3c31dXVAe0ulwufz4eqqhw4cACj0Rh0DIfDQVtbm9xZImrFxcfz
fO0rIds62t/l8bwn+UrGEg65/0BCYhIFxc9xtPWdGx6vYutvMcTFRWRsEQu+B761BoARq8LlFpe+
/ZOjfwLg039rxCam3XMX93YCyOl0cuzYMRRFwe12U1lZycqVK/Xjmc1m1q1bx+rVq0lOTmbXrl0B
odnW1kZpaSkLFiyQu0tELSUrJ2hNL1S198aenWFVfXHxD/Dw0seiJ/hiHkwgactrXHrNEdT20as1
jFgVRn+5mrS9nRgWGe+pi5uRkXFLAVRdXc2VK1eora0FoL6+nr6+PjZu3AjA+vXr8Xg8aJqGpmns
2LGD7Ozsz37rVVQwNDRER0eH3Fkiqpmycmat9mYKp+p75NGs6Am+RX+8vuiY8LPaWfum7jke0ZOr
qoFrh2VlZXi93oBtKSkp+tTS5/PhcDhuOhWeeUyv18vOnTsDpqYWi0VvKygowGQyoaoqLpcr5Pu7
XC5crutV8LJlyxgcHAxoHxoaIjMzE4DFixfT3Nystx0+fJiFCxfqFWFhYSE2m42JiQm5s0RUM698
ctZq7/Nmq/rMK74dXVPdaFZUVITdbkdRFOrq6igtLdXDKxy5ubmUlJSgKAp+v5+KigoA1qxZQ3t7
O36/H0VRsNlssx5ryZIlIUMrOTlZH5OmaUHty5cvlztJzCuhprmhqr0NPy0Pu+qLiTVET/CNWBWm
PuwP6w8an3g9d/0CtLa20tnZCUBTUxN9fX0UFxff0v7TYdTS0hIw9bxVN5sWp6Wlyd0i7hk9nX8L
q9p78EsJARXezaq+3lMnoqvii3v4azf8yIphcab+9Ud77Hf9Aly8eDFoajmX/ediYGAg5PaxsTGa
mppuuF9vb6/cSWJeUc++N2u1B6D2n+Xyx+NhVX3+/rPRFXzT1d7nwy/do5K66y+ke1TGX3/5jpzg
6fUvAEUJrjpTUlIC1wnMZsbHx0PuM/NYtysxMTHgdWpqqv71hQsXMJvNQdPfnp4ePQBnfrzFYrEw
NjamV6xCzJvgO+fj2rVrN632AH7trAsKw+mqb6apyat8ONgfmUItkt/oiFUJ+PDypd9vCWi/3NIY
8ZM7PDxMVVUVJSUlGI1G8vPzg/pYrVaam5vRNA2Hw0FiYiL79+8Hrq+n5efnYzQa0TSNqqqqOU1f
jxw5Qm1tLdXV1dTX12OxWDCZTIyOjgLQ2NiI1WrF4XDo/TIzMykvv77O0dXVpY8XYPPmzXR1dcld
JOadqclJXnX8hgpHHQDfKXiavb/bFvb+X054KOD17q01fDo1FX3BNx1+hkVGUncfY+LPb5Lwi5cD
KsJI2759O06nE1VV8fv9nDlzhtzc3IA+J06coLW1leTkZIaHh7Hb7XoFVVNTg9vt5vjx44yNjeF2
u1m1alXY79/Y2EhDQwOqqnLw4EE2bdrEvn37KC8vp7KyktOnT+Pz+fT+mqZht9txOp2Ulpbq45le
Q7TZbPp4JiYm6O7uDuuPJkJEow/O9/HXthaeKnyWH//8V3wwcI6Jjy+HtW/245/9+9rRQx40dTBi
45KHFAgh7rgbPaRg9qrxzjykQIJPCHFX6I+lyvk6OU/kYTR9NWQ/zT+I79RJBs++F52PpRJCiPlI
nsAshJDgE0IICT4hhJDgE0IICT4hhJDgE0IICT4hhJDgE0KIL87/AXLgmhXFfHNQAAAAAElFTkSu
QmCC" />

■ ディストリビューションのロールバックとリセット

ディストリビューションをアンインストールしても、tar ファイルは削除されないので、またインポートが可能である。手動で tar ファイルを削除しない限り、いつでもスナップショットとして保存した状態に戻ったディストリビューションを再インストール（ロールバック）して実行することができる。

デフォルトでインストールされた Ubuntu を完全にリセットするには

  1. `wsl --unregister Ubuntu` コマンドを実行してから
  2. `wsl --install` コマンドを実行する。

### ポートフォワード

WSL2 は Hyper-V ベースの仮想マシン上で実際の Linux カーネルを動かしている。実際の Linux カーネルは、自身のネットワークスタック（IPアドレス、ルーティング、ファイアウォールルールなど）を独立して管理するため、Windows ホストとは別の仮想マシン用ネットワークインターフェースと、それに伴う仮想ネットワークが必要になる。

この仮想ネットワークにより、WSL2 ディストリビューションは Windows ホストとは独立した IP アドレスを持つ。この IP アドレスは、Windows が起動するたびに変更される可能性がある。現在割り当てられた IP アドレスは、実行中の WSL2 ディストリビューション内で次のコマンドを実行することにより確認できる。

``` shell
hostname -I | awk '{print $1}'
```

■ Windows → WSL2 へのアクセス

最近の Windows では、WSL2 内でリッスン（待ち受け）しているポートに対して、Windows ホスト側から `localhost` を使ってアクセスできる**自動ポートフォワード機能**が標準で有効になっている。

例えば、WSL2 の Ubuntu 環境で以下のコマンドを実行し、ポート 8000 番でシンプルな Web サーバーを起動したとする。

``` shell
python3 -m http.server 8000
```

この場合、特別な設定をしなくても、Windows ホスト上の Web ブラウザから http://localhost:8000 にアクセスするだけで、WSL2 上で動作しているサーバーに接続できる。

この機能により、WSL2 の動的な IP アドレスを意識する必要がなくなり、通常のローカル開発と同じ感覚で作業を進めることができる。

【制限事項】  
もし Windows ホスト側で同じポート番号（この例では 8000 番）を使用するサービスがすでに起動している場合、`localhost` を経由した WSL2 側へのアクセスはできない。その際は、WSL2 側で別のポートを使用するか、Windows 側のサービスを停止する必要がある。

■ LAN 内の他のデバイス → WSL2 へのアクセス

WSL2 の仮想マシンは Windows ホスト内の仮想ネットワーク上にあるため、LAN に接続された他のデバイスから直接アクセスすることはできない。

他のデバイスから WSL2 上のサービスにアクセスさせたい場合は、Windows ホストの IP アドレス宛ての通信を、WSL2 の IP アドレスとポートへ転送（foward）する**ポートフォワード**の設定が必要になる。この設定は、管理者権限で起動したターミナルで `netsh` コマンドを使って行う。

``` shell
netsh interface portproxy add v4tov4 listenaddress=<転送元IPアドレス> listenport=<転送元ポート番号> connectaddress=<転送先IPアドレス> connectport=<転送先ポート番号>
```

ポートフォワード設定を削除するには、次のコマンドを実行する。

``` shell
netsh interface portproxy delete v4tov4 listenaddress=<転送元IPアドレス> listenport=<転送元ポート番号>
```

【制限事項】  
前述の通り、転送先 IP アドレスに指定する WSL2 の IPアドレスは変更される可能性があるので、Windows を再起動した際には、このポートフォワード設定も新しい IP アドレスで更新する必要がある。

### WSL2 の設定

Windows のユーザーフォルダ（`C:\Users\<ユーザー名>`）にある `.wslconfig` ファイルは、WSL2 で実行されている全ての Linux ディストリビューションに適用される構成ファイルである。

`.wslconfig` ファイルの書式は次のとおり。

``` ini
[wsl2]
キー=値
```

使用できる主なキーは次のとおり。

| キー | 説明 |
|:---|:---|
| `memory` | WSL2 仮想マシンに割り当てるメモリのサイズ。デフォルトでは Windows の合計メモリの 50% |
| `processors` | WSL2 仮想マシンに割り当てる論理プロセッサの数。デフォルトでは Windows の論理プロセッサと同じ数 |

`.wslconfig` ファイルを手動で編集するのではなく、WSL Settings の GUI を使って変更することが推奨されている。WSL Settings は [スタート] メニューにある。

![](https://learn.microsoft.com/ja-jp/windows/wsl/media/wsl-settings.png)

コンテナ
--------

### コンテナ型仮想化

**コンテナ型仮想化**（containerization）は、アプリケーションの実行環境を**コンテナ**（containers）という単位でシステムの他の部分から隔離し、ホスト OS 上で動作させる仮想化技術である。

コンテナは、システムのプロセス、ネットワーク、ファイルシステムから隔離されながらも、ホスト OS のカーネルを共有しているので、仮想マシンより軽量で高速に実行、停止することができる（下図は [RedHat 公式サイトの記事](https://www.redhat.com/ja/topics/containers/linux-kontenatoha)から引用）。

![](https://www.redhat.com/rhdc/managed-files/virtualization-vs-containers.png)

Linux ではコンテナの隔離性に**名前空間**（namespaces）を利用している。Linux カーネルは、プロセスが利用するリソース（PID、ネットワーク、ファイルシステムなど）を名前空間という単位で分離できる。例えば、新しい PID 名前空間を作ると、その中では「最初のプロセス」が `PID=1` として見える。このように名前空間は、プロセス、ネットワーク、ファイルシステムなどのシステムリソースをコンテナごとに論理的に分離し、「あたかも自分専用のリソースであるかのように見せる」役割を果たす。これにより、あるコンテナ内のプロセスが、他のコンテナやホスト OS のプロセス、ネットワーク、ファイルシステムなどに直接干渉できないようになっている。

また、Linux では複数のファイルシステムを「積み重ねて」 1 つのファイルシステムとして見せる機能がカーネルに組み込まれており、コンテナを作成するために必要な要素を**コンテナイメージ**（container image）としてまとめる際にこの機能を利用している。

コンテナイメージ内では、ベース OS、依存ライブラリ、アプリケーションコードなどがそれぞれ「読み取り専用」レイヤーとして存在している。コンテナイメージは、アプリケーションを動かすために必要なファイルシステムの状態を保持した「スナップショット」のようなものである。コンテナが実行されると、これらの読み取り専用レイヤーの上に、コンテナ固有の「書き込み可能」レイヤーが追加される。コンテナ内で行われたファイルの変更や追加は、この書き込み可能レイヤーにのみ記録され、元の読み取り専用レイヤーは変更されない。こうして差分だけ保存するので、効率がよい。

コンテナイメージを介して他人のマシンで同じコンテナを作成、実行することができる。ただし、コンテナは、ホスト OS のカーネルを共有しているため、ホスト OS 専用となる。とくに Linux のコンテナ技術が優れているため、現状では Linux コンテナが普及している。

Windows で Linux コンテナを実行するには、WSL2 などの Linux 仮想マシンが必要となる。それなら「WSL2 で Linux を動かせば十分」とも思える。しかし、WSL2 では、エクスポートして配布できる形式が、1 つの Linux ディストリビューションがまるごと入った巨大な tar ファイルとなるため、開発環境を配布しにくい。一方、コンテナなら、アプリケーション単位で分離でき、アプリケーションに必要な最小限のライブラリを入れてサイズの小さなコンテナイメージを作成できるので、開発環境を配布しやすい。実際、多くのコンテナイメージが配布されているので、大抵の開発環境はコンテナイメージを導入する形で簡単に構築できる。このため、Windows でも Linux コンテナが積極的に利用されている。

### Docker と Docker CE

[Docker](https://www.docker.com/ja-jp/) は、Docker 社が開発している、Linux コンテナを運用するための管理ツールである。

Docker では、複数のコンポーネント単位で機能が開発されている。全てのコンポーネントを含むパッケージは、一部の商用利用に有料サブスクリプション契約が必要である。主なコンポーネントは以下の通り:

  * **Docker Engine**: コンテナを作成、実行するための中心的なソフトウェア
  * **Docker CLI**: ターミナルから Docker を操作するコマンドラインツール
  * **Docker Compose**: 複数のコンテナを定義し、実行するためのツール
  * **Docker Desktop**: デスクトップ上で Docker を操作するための GUI アプリケーション
  * **Docker Hub**: クラウド上で提供されている Docker イメージの共有サービス


全ての Docker コンポーネントを含むパッケージは、公式の Docker Desktop インストーラーを使用してインストールすることができる。

Windows 版 Docker Desktop インストーラーの場合、実行すると、WSL2 が自動的に利用可能になり、docker-desktop と docker-desktop-data という 2 つの特別な Linux ディストリビューションがインストールされる。docker-desktop では、Docker Engine がインストールされ、バックグラウンドで動作する（Docker デーモン）。一方、Windows では Docker Desktop や Docker CLI などがインストールされ、WSL2 が接続する Windows 内の仮想ネットワーク経由で Docker Engine と通信する。Windows 側の Docker コンポーネントと WSL2 側の Docker Engine の連携は、**WSL Integration** と呼ばれる。下図はこの構成を表している。

``` text
┏━━━━━━━┓                ┏━━━━━━━━━┓    ┏━━━━━━━━━━━━━━━━━━━┓
┃Docker CLI    ┃WSL Integration ┃┏━━━━━━━┓┃    ┃┌────┐┌────┐┌─────┐┃
┣━━━━━━━┫―――――――→┃┃Docker Engine ┃┃―→┃│Docker  ││コンテナ││ボリューム│┃
┃Docker Compose┃←―――――――┃┃              ┃┃←―┃│イメージ││イメージ││          │┃
┣━━━━━━━┫                ┃┗━━━━━━━┛┃    ┃└────┘└────┘└─────┘┃
┃Docker Desktop┃                ┃  docker-desktop  ┃    ┃          docker-desktop-data         ┃
┃              ┃                ┗━━━━━━━━━┛    ┗━━━━━━━━━━━━━━━━━━━┛
┃              ┃                ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃              ┃                ┃                              WSL2                              ┃
┗━━━━━━━┛                ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                            Windows OS                                            ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
```

Docker コンポーネントのうち、Docker Engine、Docker CLI、Docker Compose についてはオープンソース化されている。この 3 つのコンポーネントの括りは、しばしば **Docker CE**（Community Edition）と呼ばれ、Apache license 2.0 に基づいてライセンスされている。Docker CE から Docker Hub を利用することは可能（ただし回数制限などの制約はある）。

Docker CE は Linux 環境が必要である。Windows に導入する場合、WSL2 の Linux ディストリビューション上に Docker CE をインストールすることになる。Ubuntu の場合、この構成は次のようになる。

``` text
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃┏━━━━━━━┓  ┌────┐┌────┐┌─────┐┃
┃┃Docker CLI    ┃  │Docker  ││コンテナ││ボリューム│┃
┃┣━━━━━━━┫  │イメージ││イメージ││          │┃
┃┃Docker Compose┃  └────┘└────┘└─────┘┃
┃┣━━━━━━━┫                                        ┃
┃┃Docker Engine ┃                                        ┃
┃┗━━━━━━━┛                                        ┃
┃                          Ubuntu                          ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                           WSL2                           ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                        Windows OS                        ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
```

基本的には普通の Ubuntu に Docker CE をインストールしているのと同じである。したがって、Docker CE のインストールとアンインストールの手順は、Ubuntu でも Windows でも同様となる。

Ubuntu に Docker CE をインストールする方法は、インストールスクリプトを使用する方法が簡単である。手順は以下の通り（Windows の場合、ターミナルに登録された Ubuntu プロファイルを開いて実行する）。

① `curl` コマンドでインストールスクリプトをダウンロードする:

``` shell
curl -fsSL https://get.docker.com -o get-docker.sh
```

② sudo 権限でスクリプトを実行する（Docker Desktop を推奨するメッセージが表示されるが、20 秒後に Docker CE のインストールが開始される）:

``` shell
sudo sh get-docker.sh
```

③ インストールスクリプトを削除する:

``` shell
rm get-docker.sh
```

追加の設定として、`docker` コマンドを `sudo` 権限なしで実行できるようにする（任意）。

① 新しいユーザーグループ `docker` を作成する:

``` shell
sudo groupadd docker
```

② 現在のユーザーを追加する:

``` shell
sudo usermod -aG docker $USER
```

③ グループ情報の変更を現在のセッションに反映させる:

``` shell
newgrp docker
```

動作確認をする。

``` shell
$ docker -v
Docker version 28.1.1, build 4eba377
```

【制限事項】  
WSL2 の Ubuntu に Docker CE をインストールする場合、Windows から直接コンテナを操作することはできない。基本的には、WSL2 の Ubuntu を実行してその中でコンテナを操作することになる。

Docker CE をアンインストールする方法は、以下の通り。

① Docker 関連のパッケージをアンインストール:

``` shell
sudo apt-get purge docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin docker-ce-rootless-extras
```

② 全ての Docker イメージ、コンテナ、およびボリュームを削除:

``` shell
sudo rm -rf /var/lib/docker
```

``` shell
sudo rm -rf /var/lib/containerd
```

【制限事項】  
Windows（WSL2）の場合、Docker CE をアンインストールしてもディスクの空き容量が増えるわけではない。WSL2 の仮想ディスクは、その特性上、内部でファイルを削除しても、自動的に縮小されないからである。ディスクの空き容量を増やすことが目的なら、ディストリビューションをロールバック、または完全にリセットする必要がある。

### Docker イメージ

Docker では、コンテナ作成の基礎となるコンテナイメージは「Docker イメージ」と呼ばれ、[Docker Hub](https://hub.docker.com/) に登録する形で配布されている。

Docker Hub 上では、Docker イメージは `nginx:1.25` や `python:3.13-slim` のようにリポジトリ名とタグに関連付けられて登録されている。

タグは、Docker イメージのバージョンや変種を識別するためのラベルである。`latest` タグは特別なタグで、開発者が指定する最新版のイメージを参照するためのタグである。Docker Hub やローカルに保存された Docker イメージを参照する際にタグを省略したときは `latest` タグでイメージが参照される（例: `python` → `python:latest`）。

Docker Hub 上で Docker が公式に提供している主なイメージは次のとおり。

| リポジトリ名 | 説明 | タグ |
|:---|:---|:---|
| [alpine](https://hub.docker.com/_/alpine) | 軽量ディストリビューション Alpine Linux のイメージ | `3` など |
| [ubuntu](https://hub.docker.com/_/ubuntu) | 使いやすいディストリビューション Ubuntu のイメージ | `24.04` など |
| [python](https://hub.docker.com/_/python) | Python インストールイメージ | `3`, `slim` など |
| [httpd](https://hub.docker.com/_/httpd) | Apache HTTP Server イメージ | `2.4` など |
| [nginx](https://hub.docker.com/_/nginx) | Nginx サーバーイメージ | `stable` など |
| [mysql](https://hub.docker.com/_/mysql) | MySQL データベースサーバーイメージ | `8.0` など |

Docker Hub に関する `docker` コマンドは以下の通り。

| コマンド | 機能 |
|:---|:---|
| `docker image pull NAME[:TAG]` | Docker Hub から Docker イメージをダウンロードする。 |
| `docker image push NAME[:TAG]` | 指定のイメージを Docker Hub にアップロードして登録する |

### ボリューム

コンテナは基本的に「使い捨て」であり、削除されると内部データも消えてしまう。そこで、永続データの保存先とするためのコンテナ外の場所を指定し、コンテナからその場所にアクセスできるための「マウント」機能が用意されている。

マウントには 2 種類ある。

  * **バインドマウント**: ホスト OS 側ファイルシステムの特定のパスと、コンテナ側ファイルシステムの特定のパスをバインドする。
  * **ボリュームマウント**: **ボリューム**（volume）と呼ばれる Docker が管理する特別な場所をマウントする。

ボリュームに関する `docker` コマンドは以下の通り。

| コマンド | 機能 |
|:---|:---|
| `docker volume create VOLUME` | 指定した名前のボリュームを作成する |
| `docker volume ls` | ボリュームを一覧表示する |
| `docker volume rm VOLUME [VOLUME...]` | 指定した名前のボリュームを削除する |
| `docker volume prune` | 未使用ボリュームを一括削除する |

### ネットワーク

コンテナは、デフォルトでは独立したプロセス空間、ネットワークスタック（IPアドレス、ルーティングテーブル、ポート番号空間）、ファイルシステムを持っている。このため、複数のコンテナの連携や外部との通信には、仮想的なネットワークが必要となる。

Docker は、標準で 3 種類の仮想的なネットワークドライバー bridge, host, none を用意している。

| ドライバー | 機能 |
|:---|:---|
| bridge | 独自の IP アドレスをコンテナに割り当て、ホストとは NAT（変換）を通して通信する |
| host | ホストのネットワークをそのまま使用し、コンテナに専用の IP アドレスを割り当てない |
| none | ネットワーク機能を一切持たない |

各ドライバーをそれぞれ使用する `bridge`、`host`、`none` という名前のネットワークが Docker 側で自動的に作成される（ドライバーとネットワーク名が同じ）。また、ユーザー定義のネットワークを作成することも可能である。

ネットワークに関する `docker` コマンドは以下の通り。

| コマンド | 機能 |
|:---|:---|
| `docker network create [OPTIONS] NETWORK` | 指定した名前のユーザー定義ネットワークを作成する（`-d, --driver DRIVER` オプション付きでドライバーを<br /><br />指定できる。デフォルトでは bridge が使用される） |
| `docker network ls` | ネットワークを一覧表示する |
| `docker network rm NETWORK [NETWORK...]` | 指定した名前のネットワークを削除する |
| `docker network prune` | 未使用ネットワークを一括削除 |

### コンテナの作成と実行

``` shell
docker container create [OPTIONS] IMAGE [COMMAND] [ARG...]
```

指定された Docker イメージから新しいコンテナを作成する。指定された Docker イメージがローカルに見つからない場合、Docker Hub から自動的にダウンロードする。このため、事前に `docker image pull` コマンドを実行する手間を省略できる。

コンテナの作成は、具体的には以下のような処理を行う。

  1. コンテナのファイルシステムを準備:  
指定されたイメージの読み取り専用レイヤーの上に、コンテナ固有の書き込み可能なレイヤーを作成し、コンテナのファイルシステムを準備する。
  2. コンテナ ID と名前の割り当て:  
コンテナに一意の ID を割り当て、`-name` オプションで指定した名前、または自動生成された名前を設定する。
  3. 設定の適用:  
コンテナを起動するたびに毎回実行されるコマンド（`COMMAND [ARG...]`）、ポートマッピング（`-p`）、ボリュームマウント（`-v`）、環境変数（`-e`）、ネットワーク設定（`--network`）、再起動ポリシー（`--restart`）など、指定された全てのオプション設定をコンテナに適用する。

|| オプション | 意味 |
|:---|:---|:---|
|| `--name=NAME` | コンテナの名前 |
|＊| `-h, --hostname=HOSTNAME` | コンテナに付けるホスト名 |
|◎| `-i, --interactive` | コンテナが標準入力を受け取れる状態にして、キーボード入力を可能にする |
|＊| `-t, --tty` | 擬似端末を割り当て、端末向けの見やすい出力にする。`-i` と併せて `-it` と指定されることが多い |
|| `-p, --publish HOST:CONTAINER` | ホスト・コンテナ間のポートフォワードを設定する。複数回指定できる |
|| `-v, --volume DEST:PATH` | コンテナ側ディレクトリパス `PATH` のマウントを設定する。`DEST` にはボリュームか、ホスト側ディレクトリパスを指定す<br /><br />る。複数回指定できる |
|| `--mount KEY=VAL[,KEY=VAL...]` | マウントを設定する推奨オプション。`-v` より冗長だが明示的で可読性が高い。複数回指定できる。指定できるキーは<br /><br />次のとおり<br /><br />・`type`: `bind` ならバインドマウント、`volume` ならボリュームマウント<br /><br />・`source`, `src`: バインドマウントではホスト上のファイルやディレクトリのパス、ボリュームではボリューム名になる<br /><br />・`target`, `destination`, `dst`: コンテナ内にマウントされるファイルやディレクトリのパス<br /><br />・`readonly`, `ro`: 読み込み専用としてコンテナ内にマウントされる |
|| `--network=NETWORK` | コンテナをネットワークに接続する。複数回指定できる。省略した場合、`bridge` に接続する |
|＊| `-e, --env KEY=VAL` | コンテナ内の環境変数を設定する。複数回指定できる |
|＊| `--env-file PATH` | 環境変数を記述するファイル（.env）を設定する。複数回指定できる。ファイルには各行に 1 つの環境変数を `KEY=VAL` <br /><br />の形式で記述する |
|＊| `-w, --workdir PATH` | コンテナ内の作業ディレクトリを指定する |
|＊| `-u, --user USER[:GROUP]` | デフォルトではコンテナは `root` ユーザーで動作するが、このオプションでユーザーとグループを名前で指定できる。<br /><br />UID と GID でも指定可能 |
|| `--restart` | 再起動ポリシー（コンテナが停止したときに自動的に再起動するかどうか）を指定する<br /><br />・`no`: 自動再起動しない（デフォルト）<br /><br />・`always`: コンテナが停止したら常に再起動する（手動で停止しても、Docker デーモン再起動後にまた立ち上がる）<br /><br />・`unless-stopped`: `always` に似ているが、手動で停止した場合は再起動しない（ただし Docker デーモン再起動時に<br /><br />　は再度起動する）<br /><br />・`on-failure[:N]`: コンテナがエラー終了（終了コードが 0 以外）したときのみ再起動する。オプションで最大リトライ<br /><br />　回数を指定可能（例: `on-failure:5`） |

コンテナの名前と ID は、`docker` コマンドでコンテナを指定する際に使用する。コンテナ ID は、ランダムな SHA-256 ベースの 64 文字の 16 進数であり、`docker container create` の実行結果の標準出力において最後の 1 行に表示される。実際にコンテナ ID を使用するときは、ID の完全な文字列を指定する必要はなく、一意に識別できるだけの先頭の数文字（通常は 2 〜 4 文字程度）だけで足りる。

``` shell
docker container start [OPTIONS] CONTAINER [CONTAINER...]
```

指定されたコンテナを起動する。コンテナの作成時に保存された `COMMAND [ARG...]` が実行される。そのプロセスがすぐ終了するなら、コンテナもすぐ停止する。そのプロセスが常駐型なら、コンテナは動き続ける。

上記の◎印のあるオプションは、`docker container start` コマンドにおいても、実行中に適用する設定として指定できる。

``` shell
docker container run [OPTIONS] IMAGE [COMMAND] [ARG...]
```

`docker container run` コマンドは、`docker container create` と `docker container start` の 2 つのコマンドを連続して実行するショートカットである。

`docker container create` コマンドのオプションを指定できる。このほか、以下のオプションを指定できる。

| オプション | 意味 |
|:---|:---|
| `-d, --detach` | デタッチモード |

デタッチモードの場合、コンテナ内で実行されているプロセスの標準出力と標準エラー出力を、`docker` コマンドを実行している現在のターミナルに表示させない。そのプロセスが常駐型なら、コンテナはバックグラウンドで実行され続けることになる。

``` shell
docker container exec [OPTIONS] CONTAINER COMMAND [ARG...]
```

実行中コンテナにおいて `COMMAND [ARG...]` を実行する。

上記の◎印と＊印のあるオプションは、`docker container exec` コマンドにおいても、実行中に適用する設定として指定できる。このほか、以下のオプションを指定できる。

| オプション | 意味 |
|:---|:---|
| `-d, --detach` | デタッチモード |

``` shell
docker container stop [OPTIONS] CONTAINER [CONTAINER...]
```

指定された実行中コンテナを停止する。

| オプション | 意味 |
|:---|:---|
| `-t, --time` | 待機時間を指定する。デフォルトは 10 |

以下に、`docker` コマンドの簡単な使用例を示す。この使用例では、以下の作業を事前に済ませておく。

  * 作業ディレクトリに `index.html` を配置する。
  * WSL2 側に Docker CE をインストールしている場合は、`wsl` コマンドを `--cd` オプションなしで実行し、Linux ディストリビューションを実行しておく。

`index.html` には、以下の内容を保存しておく。

``` html
<html>
  <head>
    <title>Test Page</title>
  </head>
  <body>
    <p>This is a test.</p>
  </body>
</html>
```

① `httpd` イメージから新しいコンテナを作成し、バックグラウンド実行するために、`docker container run` コマンドを実行する。ローカルにイメージがなければ、Docker Hub からダウンロードされる。

``` shell
$ docker container run --name apache_container -d -p 8080:80 --mount type=bind,source="$PWD",target=/usr/local/apache2/htdocs/ httpd
Unable to find image 'httpd:latest' locally
latest: Pulling from library/httpd
3da95a905ed5: Pull complete
71018c0d16cb: Pull complete
4f4fb700ef54: Pull complete
3b93b8c3888b: Pull complete
c5094faccdbb: Pull complete
14bacd2841b0: Pull complete
Digest: sha256:f84fe51ff5d35124e024f51215b443b16c939b24eae747025a515200e71c7d07
Status: Downloaded newer image for httpd:latest
1d94b10a80e1f5e71f4d4987995b3ff247f5dd51179288ab30e37071c5317ca9
```

標準出力の最後の行は、新しいコンテナの ID である。

`--mount` オプション指定により、`$PWD`（Linux ホストでの作業ディレクトリ）がコンテナの `/usr/local/apache2/htdocs/` にマウントされる。`-v` オプションを使用するなら、上記のコマンドは次のようになる。

``` shell
docker container run --name apache_container -d -p 8080:80 -v "$PWD":/usr/local/apache2/htdocs/ httpd
```

WSL2 の場合、バインドマウントは注意が必要。この例のように（`--cd` オプションを付けていないので） `$PWD` が `/mnt` 配下のディレクトリを指している場合、Docker のバインドマウントと WSL2 の自動マウントの 2 段構成となる。WSL2 の自動マウントはパフォーマンスが非常に悪いので、本番環境で `/mnt` 配下のディレクトリをバインドするのは推奨されない。

![](https://www.plantuml.com/plantuml/png/LP2zIWGn483xUOgym6tWoiFYmDPEn48iifleBkmciZkBrXdmxhInqKfi_04FVmeh4PzcpiF7S5nNz48WFvdvlioC6pGrjbK9fTj3XK7LnNwEoXQrox087gn1avh6gUt2st3Og1teTx3ED6EwvtZ3T0436G2BzylbxIFJXECrqmF7PwOfqnnsmGTqAWs8eHefGRYZ94Sn_nasDuQ2_rvWyVQqf9UU4r-5qmksqjPZ2xhog3ET94b7javnb72PnqmdGlb_x5DrszIw39afjJamMUw6EaSRiaOBuxUyjS7WvoCZzJcPVvoUCrrnlEjOK-XU96JLDb9CPzA6xcQsaZ1stafxlm00)

`-d` オプション指定により、ポート番号 80 番（Apache HTTP Server のデフォルトのポート番号）をリッスン中のコンテナがバックグラウンドで動作している。

`-p 8080:80` オプション指定により、Linux ホストの 8080 番ポートに送信されたデータはコンテナの 80 番ポートに転送される。これにより、ホスト側のブラウザで http://localhost:8080 にアクセス可能となり、コンテナ内の `/usr/local/apache2/htdocs/` にある `index.html` が表示される。Windows の場合は、Docker のポートフォワードと WSL2 のポートフォワード（自動ポートフォワード機能）の 2 段構成となり、WSL2 の自動ポートフォワード機能により、Windows 側で localhost アクセスが可能となっている（自動マウントされた作業ディレクトリ内の `index.html` が表示される）。

![](https://www.plantuml.com/plantuml/png/PL2nJiCm4DtlA-u7ugGHKaC6r0zWGAA80J6uiTjOJUtAlao89POOc9Zu12Hc11D_un_XaYnKVJhxPD_pktTU1PAU-aC7dTuIaaDlTYsXCbutP9o5espsSgSnnVSKFzFmaOOVV82r3eGtOs42P0nOjoOWXqJYiW368sozF6ZSUAEO_0IDz0hlRwzB5XaJ20RSRNfBFREj8yqsZZKkbgLOB4LPNFAwgAkYAhYgsCdmbIB7IuglBFB_EDdWc-yKVrDyutsMXxmFFkzS8xjSyaIjBlEMb6j2Vjwp4eNWn1tX6Kx6phAhoVhymbdiZDtNEEcsBj1iScHCr1DdiD9MyKV_0G00)

② コンテナ内で対話形式の bash シェルを起動するために、`docker container exec` コマンドを実行する。bash シェルを起動すると、コンテナ内で使えるコマンドを入力、実行できるようになる。`exit` を入力すると、bash シェルを終了してホスト側に戻る。

``` shell
$ docker container exec -it apache_container bash
root@1d94b10a80e1:/usr/local/apache2# pwd
/usr/local/apache2
root@1d94b10a80e1:/usr/local/apache2# exit
exit
$
```

`1d94b10a80e1` はコンテナ ID の先頭の 12 文字である。

③ コンテナを停止するために `docker container stop` コマンドを実行する。この例では、コンテナ ID の先頭の 12 文字でコンテナを指定した。

``` shell
$ docker container stop 1d94b10a80e1
1d94b10a80e1
```

### Dockerfile

Dockerfile と呼ばれるテキスト形式の設定ファイルを使って、既存の Docker イメージをベースにした新しい Docker イメージを作成すること（イメージのビルド）ができる。

![](https://www.plantuml.com/plantuml/png/SoWkIImgAStDuSf9JIjHACbNACfCpoXHICaiIaqkoSpFuqhBp4bLSCbFpauj0Z6valDpSY1InGfA4765nqrB7ZSlVDoyvt7J3YMklFoKLA3GviVDGDJskBcBApadlpG5hjgZD4c92s6YWgwk7GwJi8g2rACOZqkP7QgDo1m5d5OwfEQb08C30000)

``` shell
docker image build [OPTIONS] PATH | URL
```

指定したディレクトリ（相対パスも可）、または、Git リポジトリの URL にある Dockerfile に基づいてイメージを作成する。

| オプション | 内容 |
|:---|:---|
| `--tag, -t <リポジトリ名>:<タグ>` | 作成するイメージにリポジトリ名とオプションのタグを付ける。タグを省略すると、デフォルトで `:latest` が使用される |
| `--file, -f <ファイル名>` | Dockerfile を指定。デフォルトは、`'PATH/Dockerfile'` |
| `--build-arg <変数>=<値>` | ビルド時の変数を設定する |
| `--no-cache` | すべてのレイヤーをキャッシュせずに最初からビルドする |

Dockerfile の書式は次の通り。

``` dockerfile
# コメント
命令 引数
```

**命令**（instruction）は大文字と小文字を区別しない。ただし、引数と区別をつけやすくするため、慣例として命令は大文字で記述される。

ビルドでは Dockerfile 内の命令が 1 つずつ実行され、必要な場合にはビルドイメージ内にその処理結果が確定的に書き込まれる。

主な命令は以下の通り。

| 命令 | 説明 | 例 |
|:---|:---|:---|
| `FROM` | ベースとなるイメージを指定する（必須）。Dockerfile の先頭に記述する | `FROM ubuntu:22.04` |
| `ARG` | ビルドオプション `--build-arg` で受け渡される変数を宣言する。デフォルト値を設定で<br /><br />きる。`ARG` だけは `FROM` よりも前に記述できる | `ARG user1`<br /><br />`ARG buildno=1` |
| `ENV` | 環境変数を設定する | `ENV NODE_ENV=production` |
| `WORKDIR` | 命令のイメージ内作業ディレクトリを設定する | `WORKDIR /app` |
| `COPY` | ホストOS のファイルやディレクトリをイメージ内にコピーする | `COPY requirements.txt .` |
| `ADD` | `COPY` と似ているが、URL からのファイルのダウンロードや、tar ファイルの自動展開など<br /><br />の機能も持っている | `ADD https://example.com/latest.tar.gz /tmp/` |
| `RUN` | イメージ構築中にコマンドを実行する | `RUN apt-get update && apt-get install -y curl` |
| `ENTRYPOINT` | コンテナ実行時に実行されるコマンドと引数を指定する | `ENTRYPOINT ["echo", "Hello, Docker!"]` |

ビルド前に、Dockerfile 内のコメントと、命令よりも前の空白は削除される。以下のような例では、先頭の空白は保持されないため、どちらも同じものである。

``` dockerfile
        # コメントと空白は削除されるのでFROMの前に記述できる
   FROM ubuntu
```

``` dockerfile
# これはコメント行です
FROM ubuntu
```

引数を複数行で記述するときは、行末を `\` でエスケープする必要がある。このとき引数の空白は保持される。そのため、以下の例では、先頭に空白が指定した通り `" hello world"` を表示する。

``` dockerfile
FROM ubuntu
RUN echo "\
 hello\
 world"
```

`ARG` で宣言した変数の値は、`${varname}` の形式で参照される。例えば、次の Dockerfile は、イメージのタグが `--build-arg` ビルドオプションで渡される。

``` dockerfile
ARG TAG
FROM ubuntu:${TAG}
```

Python 環境を構築するための Dockerfile は次のようになる。

``` dockerfile
FROM python:3
RUN apt-get update
RUN pip install --upgrade pip
COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt
```

カレントディレクトリにあるこの Dockerfile から `mypython` というリポジトリ名でイメージをビルドするには、次のコマンドを実行する。

``` shell
$ docker image build -t mypython .
[+] Building 61.3s (10/10) FINISHED                                                                      docker:default
 （中略）
 => exporting to image                                                                                             0.6s
 => => exporting layers                                                                                            0.6s
 => => writing image sha256:f419d39467d3c1509fb3ddbb33d432adf34817205d33d9548c63a1c2af94bb5f                       0.0s
 => => naming to docker.io/library/mypython                                                                        0.0s
```

標準出力の最後から 2 番目の行にある「sha256:」以降の文字列は、新しい Docker イメージの ID である。イメージ ID は、イメージの内容を基にした SHA-256 ハッシュ値としての 64 文字の 16 進数である。`docker` コマンドではイメージ ID で Docker イメージを指定できる。その際には、コンテナ ID と同様に、一意に識別できるだけの先頭の数文字（通常は 2 〜 4 文字程度）だけで足りる。

Docker は、ビルドで生成されたか、Docker Hub からローカルに保存されたかを問わず、ローカルにある全てのイメージに ID を設定して管理する。

Docker が ID でイメージを管理するのは、リポジトリ名とタグが不変ではないからである。例えば、既存のイメージと同じリポジトリ名とタグで新しいイメージをビルドしたとき、古いイメージはリポジトリ名とタグを失う。このように、どのリポジトリ名やタグにも関連付けられていない Docker イメージのことを**タグなしイメージ**（dangling image）という。

Dockerfile からイメージをビルドする過程で中間生成物としてのイメージも作成され、**中間イメージ**（intermediate image）と呼ばれる。中間イメージは、ビルドのキャッシュとして再利用されるため、次回以降のビルドを高速化する。通常はユーザーが直接使うことはない。

### Docker Compose

Docker Compose は、1 つのアプリケーションを構成する各コンポーネントを **サービス**（service）として **Compose ファイル**に一元的に定義し、その定義に基づいてコンテナを起動する。これにより、単一コマンドでアプリケーション全体をまとめて実行・停止・管理できるツールである。

例えば、Web アプリケーションは、「アプリケーション本体」「データベース」「頻繁にアクセスされるデータのキャッシュ」といったコンポーネント単位で機能が開発されることが多い。Docker Compose は、それらを `web`（または `app`）、`db`、`cache` のような名前のサービスで管理する。

通常、1 サービス ＝ 1 コンテナであるが、負荷分散などの目的で、同じ設定のコンテナを複数実行する（スケーリング）ことも可能である。

![](https://www.plantuml.com/plantuml/png/VP2nIiSm48VtUugZUrv1_g2Z3kee3ZLtRIDf8csAYBZamC5Td1L1q4awkFWszoAc25f49Ldk-7vt7xSPKpsagVVWkKcO8WwkxHAI6zWc5mCqpZDMsx4_YoEZogtcUysFAYyLM1ydGiqFAdSgpvfVLTurlwaygSnu2NQ91Gu9p_car25I6RPpMeSsbjoiKlwroaq5LokOljczYyFztNzGMzkEl-W39ZV-fClxtH-DCLj_wiEYR8uwHyJXs1Gfe7NZSpLiE52vtmS0)

Compose ファイルは、YAML 形式で記述する。トップレベルでマッピングを次のように定義する。

| キー | データ型 | 値 |
|:---|:---|:---|
| `version` | 文字列 | Compose ファイルのバージョン（非推奨） |
| `services` | マッピング | 各サービスの定義（必須） |
| `networks` | マッピング | ネットワークの定義（任意）。ネットワーク定義の有無にかかわらず、常に `<プロジェクト名>_default` という<br /><br />名前のユーザー定義 bridge ネットワークが 1 つ自動生成される |
| `volumes` | マッピング | ボリュームの定義（任意） |

`services` キーには、サービス名をキーとして、サービスを定義するマッピングを記述する。

■ サービスが Dockerfile よりイメージをビルドする場合、`docker image build` コマンドの引数とオプションに対応する、以下のキーを持つマッピングを記述する。

| キー | データ型 | 値 |
|:---|:---|:---|
| `build` | 文字列 or マッピング | Dockerfile があるディレクトリへの相対パス、またはビルドオプションを表すマッピング |
| `image` | 文字列 | `--tag` オプションと同様で、作成するイメージにリポジトリ名とオプションのタグを付ける |
| `command` | 文字列 or シーケンス | `ENTRYPOINT` 命令の追加の引数 |

`build` キーに指定するマッピングは、以下のキーを持つことができる。

| キー | データ型 | 値 |
|:---|:---|:---|
| `context` | 文字列 | `PATH` 引数と同様で、Dockerfile の相対パスを指定するための基準となるディレクトリ |
| `dockerfile` | 文字列 | `--file` オプションと同様で、Dockerfile を、`context` を基準とする相対パスで指定 |
| `args` | マッピング | `--build-arg` オプションと同様で、変数名をキーとするマッピングで変数の値を指定 |

``` yaml
services:
  ubuntu:  # 'ubuntu' という名前のサービスを定義
    build:
      context: .  # カレントディレクトリの Dockerfile を使ってイメージをビルド
      dockerfile: Dockerfile  # Dockerfile のファイル名を指定
      args:
        TAG: "22.04"  # 変数 'TAG' を設定
    image: "plain:22.04"
```

■ サービスが既存の Docker イメージをそのまま使う場合、以下のキーを持つマッピングを記述する。

| キー | データ型 | 値 |
|:---|:---|:---|
| `image` | 文字列 | Docker イメージを指定する |
| `command` | 文字列 or シーケンス | イメージに既に適用されているコンテナ起動時に実行されるコマンドの設定を上書きする |

``` yaml
services:
  app:  # 'app' という名前のサービスを定義
    image: alpine  # alpine:latest のイメージを使用
    command: ["echo", "hello", "world"]  # シーケンスでコマンドを設定。文字列で "echo hello world" と設定するのと同じ
```

■ 上記の 2 つの場合に共通して、以下のキーを持つマッピングを記述できる。いくつかは、`docker container create` のオプションに対応する。

| キー | データ型 | 値 |
|:---|:---|:---|
| `hostname` | 文字列 | コンテナに付けるホスト名。`-h` オプションに対応 |
| `stdin_open` | 真理値 | `true` を指定するとき、標準入力を受け取れる。`-i` オプションに対応 |
| `tty` | 真理値 | `true` を指定するとき、擬似端末を割り当てる。`-t` オプションに対応 |
| `ports` | 文字列 | ホスト・コンテナ間のポートフォワードの設定。`-p` オプションに対応 |
| `volumes` | 文字列 | マウントの設定。ボリュームマウントを指定する場合は、トップレベルの `volumes` キーに記述が必要。`-v` オプションに対応 |
| `networks` | 文字列 | ネットワークの設定。トップレベルの `networks` キーに記述が必要。省略した場合、`<プロジェクト名>_default` に接続す<br /><br />る。`--network` オプションに対応 |
| `environment` | マッピング | コンテナ内の環境変数の設定（キーは環境変数名）。`-e` オプションに対応 |
| `env_file` | 文字列 or シーケンス | 環境変数を記述するファイル（.env）（のシーケンス）の指定。`--env-file` オプションに対応 |
| `working_dir` | 文字列 | コンテナ内の作業ディレクトリ。`-w` オプションに対応 |
| `user` | 文字列 | ユーザー（とグループ）。`-u` オプションに対応 |
| `restart` | 文字列 | 再起動ポリシー。`--restart` オプションに対応 |
| `depends_on` | シーケンス | サービス間の依存関係を定義する。シーケンスで指定したサービスが全て起動してからこのサービスを起動する |

サービスを実行すると、`<プロジェクト名>_<サービス名>_<番号>` という名前でコンテナが作成される。Docker Compose を使用する限りでは、サービスが管理の対象となって、コンテナ名は使用されず重要ではない。

Docker Compose は、Docker CLI の公式プラグインとして組み込まれている。コマンドの構文は、次のとおり:

``` shell
docker compose [-f <ファイル名>...] [オプション] コマンド [サービス...]
```

共通のオプション `-f` または `--file` は、Compose ファイルを指定する。デフォルトでは、カレントディレクトリの `compose.yaml`（推奨）または `docker-compose.yml` が使用される。`-f` オプションは複数回指定できる。

| コマンド | 機能 |
|:---|:---|
| `docker compose up` | Compose ファイルで定義された、指定（省略なら全て）のサービスを実行する。コンテナ、ネットワーク、ボリュームは自動作成される |
| `docker compose exec` | `docker container exec` と似ているが、コンテナではなくサービスを指定する |
| `docker compose down` | Compose ファイルで定義された、指定（省略なら全て）のサービスを終了する。関連するコンテナ、ネットワークを削除する。<br /><br />`--volumes , -v` オプションを付けると、Compose ファイルで定義されたボリュームも削除する |

`docker compose up` コマンドは、以下のオプションを受け付ける。

| オプション | 機能 |
|:---|:---|
| `--detach , -d` | デタッチモード。サービスをバックグラウンドで実行する |
| `--build` | コンテナを起動する前に、サービスのイメージを強制的に再作成する |
| `--scale SERVICE=NUM` | SERVICE を NUM 個のコンテナにスケールする |

### 名前解決

bridge ネットワークに接続されたコンテナ同士は、自動的に割り当てられる IP アドレスを使って通信する。しかし、毎回 IP アドレスを確認しなければならないのは不便である。そこで Docker では、IP アドレスを直接指定せずに名前で通信できるようにする仕組みが用意されている。この仕組みは**名前解決**（name resolution）と呼ばれる。

■ Docker における名前解決

`docker network create` コマンドでユーザー定義の bridge ネットワークを作成すると、コンテナ名による名前解決が可能になる。

例えば、次のシェルスクリプトは、ユーザー定義ネットワーク `mynet` を作成し、`mynet` に接続する 2 つのコンテナ `container-a` と `container-b` を起動する。2 つのコンテナは、起動してから 5,000 秒間 sleep するだけのコンテナである。最後に `container-a` 内に入って `ping` を実行する。

``` shell
# ユーザー定義ネットワークを作成
docker network create mynet

# alpine イメージから 2 つのコンテナを作成して実行
docker run -d --rm --network=mynet --name=container-a alpine sleep 5000
docker run -d --rm --network=mynet --name=container-b alpine sleep 5000

# container-a コンテナ内で ping を実行
docker exec container-a ping -c 3 container-b
```

このシェルスクリプトを実行すると、`container-a` 内から `container-b` に名前解決できることがわかる（最初の 3 行はネットワーク ID とコンテナ ID が出力されたもの）。

``` shell
$ ./sample.sh
ae8dca8be96734244c9e41270cd82925fbfda058d8d0c0be6b7900c79490d387
b37b37cc0315fdafa453d04ea27b44140e1c226758f0acd1cf86287792a5f847
63ab2491f542a47c5e60b57a7a98a3046fd3738cef0bd02b643b54f010410e8e
PING container-b (172.19.0.3): 56 data bytes
64 bytes from 172.19.0.3: seq=0 ttl=64 time=0.138 ms
64 bytes from 172.19.0.3: seq=1 ttl=64 time=0.091 ms
64 bytes from 172.19.0.3: seq=2 ttl=64 time=0.062 ms

--- container-b ping statistics ---
3 packets transmitted, 3 packets received, 0% packet loss
round-trip min/avg/max = 0.062/0.097/0.138 ms
```

デフォルトの `bridge` ネットワーク（`bridge` という名前の bridge ネットワーク）では、コンテナ名による名前解決はできない。通信には IP アドレスを指定する必要がある。

■ Docker Compose における名前解決

`docker compose up` を実行すると、Docker Compose は自動的にユーザー定義ネットワークとして `<プロジェクト名>_default` を作成し、デフォルトでは全てのサービスはこのネットワークに接続され、サービス名で名前解決が可能になる。

次の Compose ファイルは、2 つのサービス `client` と `db` を定義する。`docker compose up` を実行すれば、`client` から `db` に名前解決できることがわかる。

``` yaml
services:
  client:
    image: alpine
    command: sh -c "apk add --no-cache iputils && ping db"
    depends_on:
      - db
  db:
    image: "mysql:8.0"
    environment:
      MYSQL_ROOT_PASSWORD: example
```

サービス名に名前解決されるため、`client` から `db:3306` でデータベースに接続できる（3306 番ポートは MySQL データベースサーバーがデフォルトで使用するポートである）。なお、`client` と `db` からはそれぞれ異なるコンテナが起動するので、`client` から `localhost:3306` でデータベースに接続できないことに注意する。また、MySQL 固有の問題として、デフォルトでは外部（`localhost` 以外）のクライアントから接続ができない。以下のサイトで解決法が解説されている。

  * [MySQL に外部(localhost以外)からアクセスを許可する - Oji-Cloud](https://oji-cloud.net/2024/02/22/post-8180/)

`db` が複数コンテナにスケールされる場合、各コンテナの名前には番号が振られ、`db` に接続するときは名前の番号順に接続可能なコンテナを見つけて自動的に名前解決される。

### ネットワークモード

Docker Compose には、特殊なケースでのみ利用される機能として、サービスの「ネットワークモード」がある。ネットワークモードは、通常利用されるユーザー定義の bridge ネットワークではなく、Docker が自動的に作成するデフォルトの `bridge`、`host`、`none`、あるいは既存コンテナのネットワーク名前空間を利用する機能である。サービス定義において `network_mode` キーで設定する。

| モード | 説明 |
|:---|:---|
| `bridge` | デフォルトの `bridge` を使用 |
| `host` | デフォルトの `host` を使用 |
| `none` | デフォルトの `none` を使用 |
| `service:<サービス名>` | 他のサービスとネットワーク名前空間を共有 |

ネットワークモードを設定したサービスは、Docker Compose が自動で作成するユーザー定義ネットワークには参加しないので、サービス名での名前解決が効かなくなる。スケールにも対応できない。

例えば次の Compose ファイルでは、`app` サービスが `db` サービスとネットワーク名前空間を共有する。

``` yaml
services:
  app:
    image: alpine
    command: ping localhost
    network_mode: "service:db"
  db:
    image: "mysql:8.0"
    environment:
      MYSQL_ROOT_PASSWORD: example
```

完全に同じネットワーク空間を使うため、`app` サービス内から `localhost:3306` でデータベースに接続できる。

MySQL のようにデフォルトで `localhost` からしか接続を受け付けないサービスでも、ネットワークモードでネットワーク名前空間を共有すれば、クライアントから `localhost` で接続できるため、開発環境を簡単に構築できる。

一方、ネットワーク名前空間を共有するとコンテナの分離性が失われるし、スケールにも対応できないので、本番環境では「サービス名による名前解決」が推奨される。

### コンテナとイメージの管理

コンテナとイメージを管理するコマンドは、以下の通り。

| コマンド | 機能 |
|:---|:---|
| `docker container logs` | コンテナのログを表示 |
| `docker container ls [OPTIONS]` | 実行中コンテナを一覧表示（`--all`, `-a` オプション付きで全コンテナを表示） |
| `docker container rm CONTAINER [CONTAINER...]` | 指定された停止中コンテナを削除 |
| `docker container prune` | 停止中コンテナを一括削除 |
| `docker image ls [OPTIONS] [REPOSITORY[:TAG]]` | ローカルにあるイメージの一覧表示（`--all`, `-a` オプション付きで中間イメージも表示） |
| `docker image rm IMAGE [IMAGE...]` | 指定されたイメージを削除 |
| `docker image prune` | タグなしイメージを一括削除（`--all`, `-a` オプション付きで中間イメージも削除） |
| `docker system df` | ディスク使用量を表示 |
| `docker system prune [OPTIONS]` | コンテナ、ネットワーク、タグなしイメージ、中間イメージを一括削除（`--all`, `-a` オプション付きで未使<br /><br />用イメージも削除、`--volumes` オプション付きでボリュームも削除） |

`docker container ls` や `docker image ls` コマンドでは ID が表示されるが、先頭の 12 文字までである。

``` shell
$ docker container ls
CONTAINER ID   IMAGE     COMMAND              CREATED          STATUS          PORTS                                     NAMES
1d94b10a80e1   httpd     "httpd-foreground"   42 minutes ago   Up 42 minutes   0.0.0.0:8080->80/tcp, [::]:8080->80/tcp   apache_container
$ docker image ls
REPOSITORY   TAG       IMAGE ID       CREATED      SIZE
httpd        latest    90f191b9781e   8 days ago   148MB
```

`docker image ls` コマンドは、タグなしイメージのリポジトリ名とタグを `<none>` と表示し、デフォルトでは中間イメージを表示しない。

Dev Containers
--------------

コンテナに関係する VS Code 拡張機能の情報をまとめる。

### WSL 拡張機能

VS Code の [WSL 拡張機能](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-wsl)をインストールすると

  * Windows 上の VS Code から、WSL ディストリビューションのファイルシステムにアクセスして、仮想ディスク内の Linux ファイルを直接編集することができる。
  * WSL ディストリビューション上のコマンドラインから、VS Code を起動し、マウントされた Windows 側のプロジェクトを開くことができる。

このような連携機能は、VS Code を「クライアント・サーバー」アーキテクチャに分割することで実現されている。

WSL 拡張機能は、WSL ディストリビューション側に VS Code Server を自動的にインストールし、Windows 側の VSCode を、VS Code Server と通信して画面描画のみを担当するクライアントにしてしまう。エディタの基本機能を担当する VS Code Server が WSL ディストリビューション側にインストールされるので、Windows で起動した VS Code の画面上で WSL ディストリビューションのファイルシステムにアクセスすることが可能となる。

また、VS Code Server は Windows 側の VSCode を呼び出すことが可能であり、WSL ディストリビューション上のコマンドラインで `code .` を入力して VS Code Server を直接起動した場合、Windows 側で画面描画を行う。このときカレントディレクトリが `/mnt/<ドライブ名>` 以下のディレクトリであれば、自動マウントを介して Windows 側のプロジェクトを開くことになる。

![](https://www.plantuml.com/plantuml/png/ZPAzIWGn58NxFCKb_Rw0XIpORZUWXLf44tH0iv39gY0szuAAhOYTF_WZA5OM8We-p7aHasM5mHbdfvrpxZdVOJAieWbnihkZDasmj59wEzwlw52PBUSZwUM2biRMQMLDD1kcSgHddZKFkG3VWkyXB-1ty0Va67AZrT4qBLcB4K5Em7TJyHJo33c7F42-yeKyGTvqeuqA5_PSg9UEIZyveFyRPm7Wxnmjrv2px95Bo2luCpapLGzs6YpERQE5f5s17nDzMgL-kVDnSyJV3tLJcze3e3MpUtoNsisTrDNjqbhdrmUz_lxGUPjUr0y0)

WSL 拡張機能を実行するには

  * <kbd>Ctrl</kbd> + <kbd>Shift</kbd> + <kbd>P</kbd> キーを同時に押してコマンドパレットを開き、`wsl` を入力して、「Connect to WSL」（WSL への接続）を選択する。
  * または、VS Code 画面の左下にある「><」ボタンを押して開くコマンドパレットから「Connect to WSL」（WSL への接続）を選択する。

<img src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAMEAAABUCAMAAAAMN6NeAAAAAXNSR0IB2cksfwAAAARnQU1BAACx
jwv8YQUAAAAgY0hSTQAAeiYAAICEAAD6AAAAgOgAAHUwAADqYAAAOpgAABdwnLpRPAAAAH5QTFRF
YGBgPT09MzMzJSUmRkNCgoKDpaWlnZ2dsrKyra2tT09PKyssdHR0ODg4kpKSJCQlv8XDzs7O09PT
6vH4OjYzLEc9KUhcUpx/KopnFYFcFYNdA3u6A3zQAXrMAHbLIYjSQZjYXqfe/v//0+bzuNfwm8fq
frjlhbynI3JWEm2qk/JTtQAAAAlwSFlzAAALEwAACxMBAJqcGAAABQ5JREFUaN7tmQlvqzgQgA3L
HWPheLuhEWDAx3v9/39wx+bIUUjSBilBYqQQ1+aYby4PKUrcZ2T3z8sFbQQbwUawEWwEG8FG8P4E
jg+C10yQxHEUk5USOOaAYpDw9PeaCHCAdu4OG4LAMeEUeusiMKr7ThIaghg7CAZBsiYCa/s4iC8k
8FZEQOIpSVZEkIYTAHi3pjxA3wFIuqpM3vVxFBASRvMx9LYEO9fr7e7sdn1ax767mihyECZdGSIX
IRWSCdq3JEDRVfEZ8zoKV0YQDDN+fD2zEoLR4n60UoLYu97gVkKQYBJOZXJAfLSe/SDpK2h6ApjZ
0V4vM901GSooCeNbO1r6ernpg3PxV0WQTjWnaE0E/lRzPfmK894E4xtOsD4Ci0C84S3TQ8HcW+bb
Erh+gKFU+sObvkeCldUi6FB3D/3acvPew/p4HC9yhxNO5w6rixGciurNX7xON3Ku7+wg38eeeUR/
TBLciZdg1D0cew5C3chx+sVFCRyfEIIfIwiuHp1EdL+nGXLTaG+UjPYY7ztBZE+s3WEloRTDMKNJ
ktlFvCiBbRrcxwhAh4sIimiEUUizJI2oJaBQEXBAQzCzPxBQQwCndAQZwmhhH9yXMwJ2gYCsYmlM
iTsSwDHc+6D5JQENBgJnyUz+OUFI6Zn9CA3MIgb1vhGkFwRRSP2ewPxU7ryOgDh76pwRkI4gvktg
wscSgDs6z72KgDD/9KdvfeDC17/3CEzE9XmQJMkLfZAxcpUHrqlAPuSC76ZOZjlGAnPtQAAOo10e
LL4f/CyTyfnT3YBmPpSezINIogRGkXNGENt/Dg0E4KAxD14YRf7ljb1gDGtiR3bn6gnMBI13hiCL
rMeo1+UBxS8jQN9vjfutN02w39f5BCf2iBDUfrsn43FPRkZeuB+8XWe3EWwEG8FGsBH8iODw91H5
/HpLQYfj54PyVb6lbAQbwcsIipUTcCnU0gxVtThB/meOoNKSS/3MU7+pX2uldF3W9YIEuWiz4wyB
bKpGVSXXmi8CwGVrhGmmlyM4ilYcZn0gtGhKLpQS9+7/SLBx0TKltQII+TzBMR88wA6zeVCIVlZl
LauC3XY7sD6gg2wFh3isWduK4lmCYyTyzgPfAM5rkSXgrGnsE2vOIYDhY2KK142lgi8OccbMpB2X
HE6bUrABAHuFVuJ5gpy1BiHftyy/RwCPZo0xtFCMMS0YE8ABX0yVpWaCCa7gdk1llsHKWogpl1W6
Vc+Uo+soOowI4nCHoAICbglAMQVq1EyDmrzULecwNB/NSjvNmapUq4sJE5v5YaxV/XwmdwhHg5Df
IuC8YV3igQ8gKVoONUpV3MRKW4PGvOBgd1aVSkCYwWAm7as+7IwATPV8LfrTttmnRYiO8wQQEW3n
gp6A9QQSoqetSwgqoQtLACM4mxVKTGtX8CF+oEI0C/hAdMY/5vGtKDLlu0vASwIh6oK3xqi1gsgx
BFIWRqo5Ag5ZIs0V1ZDTTxH0qXynqwACVfM+qM8JStiTCg1RZFSR0uYB1COYk7MEBXhTmlql2S9c
MFGL9vm9vqhSqlangL3IA8WkkKwuJJOSNcaqdSFgDsZzBMaj4AVpzLJALYrmPDDRVYwVpDEdBnik
aSowtubQ4ZRa2rqi5TCumrmWAVoim1VQeBcgyPP7vWnV2K5iMeHc7INN/bt293fvB41aEGB7R9sI
Xk+Qr57g42GEr7J4R0Hux0fymPz3nvI/k8hk+OJ2tP8AAAAASUVORK5CYII=" />

既定の WSL ディストリビューションに接続して VS Code Server を呼び出すので、VS Code の画面上で Linux フォルダを開くことができる。

コマンドパレットから「Connect to WSL using Distro...」（ディストリビューションを使用して WSL に接続...）を選択すると、既定以外の WSL ディストリビューションに接続することもできる。

接続後、再び「><」ボタンを押して開くコマンドパレットから「Close Remote Connection」（リモート接続を終了する）を選択すると、VS Code Server との接続が終了する。

VS Code Server は独自に拡張機能を管理する。WSL 拡張機能を実行し、アクティビティーバーから「拡張機能」サイドバーを開くと、ローカル（Windows 側）と WSL ディストリビューションの間で分割された拡張機能の一覧が表示される。一部の拡張機能は、各 WSL ディストリビューションに個別にインストールする必要がある。

### Container Tools 拡張機能

VS Code の [Container Tools 拡張機能](https://marketplace.visualstudio.com/items?itemName=ms-azuretools.vscode-containers)をインストールすると

  * アクティビティーバーにアイコン（下の画像）が追加され、GUI でコンテナや Docker イメージを操作、管理することができる。
  * Dockerfile と Compose ファイルを編集するとき、入力補完される。

<img src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAADEAAAAxCAYAAABznEEcAAAAAXNSR0IB2cksfwAAAARnQU1BAACx
jwv8YQUAAAAgY0hSTQAAeiYAAICEAAD6AAAAgOgAAHUwAADqYAAAOpgAABdwnLpRPAAAAAlwSFlz
AAAOwwAADsMBx2+oZAAABRpJREFUaN7tmF9sS30Yxz97O8p6glkjy6zG2GILEU0TJARJd6bOcY65
kYzd7IboTbNkly65dke4JxJO1387F93IEFRSi6SypTFiqYicmqElRPW92nnVeKd0431znsunvz/9
/H6/8zzf56mqr68v8h+3v/gfmAVhQVgQFoQFYUH8m1XP5+IOhwNZljl8+DC1tbUAxGIxNE3j8ePH
Fdunaj5kR1tbG4qisHfvXux2+zfHpFIpBgYGGBkZ4dOnT38GhN1ux+v1oqoqLS0tpt8wDMLhMKlU
ipUrVyJJEm632/z91atXxGIxQqEQ2Wz290A0Njaiqio+nw+HwwFAoVDg7t27RCIR7t27R7FYnDXn
wIED7Nu3j2XLlpXMCQaDJJPJhYHYvXs3qqqydetW05fNZhkcHCQSifzwqc7c3qZNm0xfJpMhGAyi
aVrlIZxOJ4qisH//furq6kx/IpEgHA5z+/btn77R5uZmFEVBFEWWLl0KwPj4OH19fbx///7Xo1N9
fT1+v5+dO3eavunpaaLRKNFolBcvXvzyN1UoFGb5Nm7ciCRJXLly5dcgXC4XZ8+eNd97Npvl3Llz
DA8PVySSdXZ2oigK7e3tJf5cLocgCCU3/tMQHR0dJsDMkzp69CiCIDA0NEQ+ny/7j69Zs4ZDhw7h
9XpL1jYMg1gsRiwW49ixY3i93soku0WLFgEwOTlJoVBg3bp1rF27lkAgwIkTJ4jH4wSDQSYmJubc
TBRFZFlm8+bNJf6bN2+i6zp37tz552Otqqp8xk6n05w6dYrW1lY6OzsRRRFBEJAkCUmSSKfThEIh
hoeH+fDhgzlv9erVyLKMLMsIglASySKRCNFolKmpKQRBQFVVQqEQwKywXFHZkU6nSafTPHz4kJMn
T5r+1tZW+vv78fv9xONxUqkUPp+vJLF9K5KtX7+eqakp2tvbCQQCJJNJMpnMwmqnTCbDmTNnUFWV
Xbt2UVNTg6qqqKpacuq6rhOJRDAMgw0bNgDQ0tLC+fPn8fv95vOx2WwLLwCLxSLJZJJkMsny5cuR
JInu7m4cDgfj4+NcunSJGzdumOPr6uq4cOEC/f395HI5ABYvXvznSPHXr19z8eJFJicnAdB13QTQ
NA23220Gii+j0rzXE+VEie/ZkiVLqK2txeVylaz75dpf71POvlZlZ0EsJEQ5SaecNYrFYon/+fPn
5PN53rx5M//JrpL29u1bAD5+/MjTp0+RZRmALVu2mLnk3bt3fx7EjOT+/Pkzz549o7e3lydPnlBT
U0NHRwddXV00NTWZ43+kRpkTYnp6GgCPx0Nvb6+ZdcuxGS1VKBQwDINAIGAKRpvNRl9fH6IoljQV
bt26xeXLl39IWM4Jce3aNY4cOcKKFSvo6emhp6eHRCJRdiF0+vRpRkdHAXjw4IF56m1tbeaYly9f
Mjg4SDgcLqtpMCeEYRh0d3cjiiJdXV00Njaybdu2ki6H3W4vUa7fsng8TkNDA8ePH8fn85kNAoDR
0VFCoRAjIyPz1zzL5XJomoamaXg8Hg4ePMj27dux2WysWrWKq1evous6oVBolgK12Wzs2LEDRVHw
eDxmJs7n89+ds2AtG6fTiaqqKIoy61RdLhdOp5P79+/T1NRUUmI+evSIgYEBhoaG5ry9Bes7VVdX
s2fPnlltl68/7OvXrxMOhxkbG6t4xKtoG7O5uRlRFHG73TQ0NDA2NkYikUDX9Z+qxX9rL9bSThaE
BWFBWBAWhAVRYfsbfIYYLWc+IOwAAAAASUVORK5CYII=" />

WSL ディストリビューションに Docker CE をインストールした場合、Container Tools 拡張機能は WSL 環境の VS Code（WSL 拡張機能で接続した VS Code Server）にインストールする必要がある。なぜなら、Windows 環境の VS Code にインストールされた Container Tools 拡張機能では、WSL2 側の Docker CE を検知できないからである。したがって、WSL 拡張機能を実行して WSL 環境の VS Code に接続してから Container Tools 拡張機能を使用するという使い方になる。

### Dev Containers 拡張機能

VS Code の [Dev Containers 拡張機能](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-containers)をインストールすると

  * Dockerfile や Compose ファイルを一から書かずに素早く開発環境を構築できる。
  * VS Code からコンテナ内のファイルシステムにアクセスし、その状況で VS Code の全ての機能が利用可能である。

VS Code とコンテナの連携は、WSL 拡張機能と同様の「クライアント・サーバー」アーキテクチャにより実現される。ただし、Dev Containers 拡張機能のほうは、コンテナ内に VS Code Server をインストールする。コンテナ内の VS Code Server は、コンテナ内にインストールされたプログラミング言語やツールを使用でき、また、コンテナ内で起動したアプリケーションがリッスンしているポートを検出して自動的にそのポートをローカル側に転送し、`localhost` からアクセスできるようにする。ローカル側の VS Code はコンテナ内の VS Code Server と通信し、画面描画のみを担当している（画像は [公式ドキュメント](https://learn.microsoft.com/ja-jp/training/modules/use-docker-container-dev-env-vs-code/3-use-as-development-environment)から引用）。VS Code Server がインストールされたコンテナは、**開発コンテナ**（development container）と呼ばれる。

![](https://learn.microsoft.com/ja-jp/training/modules/use-docker-container-dev-env-vs-code/media/2-architecture-containers.png)

Dev Containers 拡張機能は、バックエンドとして Docker CLI に依存する。Windows で WSL2 側に Docker CE をインストールした場合、Windows からは `docker` コマンドを実行できないので、以下のような設定が必要である。

  1. Windows 環境の VS Code 上で <kbd>Ctrl + ,</kbd> を入力し設定を開く。
  2. `dev.containers.executeInWSL` を検索。
  3. 「Dev › Containers: Execute In WSL」をチェック。

<img src="data:image/png;base64,
..." />

【制限事項】

Windows の場合、VS Code の機能とコンテナがシームレスに連携され、ユーザーはコンテナの存在を意識することがなくなるため、「Windows 環境で作業している」あるいは「WSL2 の Linux 環境で作業している」と錯覚しがちである。

開発コンテナ環境の VS Code は、Windows のファイルシステムにも、WSL2 の Linux ファイルシステムにも基本的にはアクセスできない（バインドマウント経由のアクセスは可能）。混乱しがちなので注意が必要。

WSL2 の Linux に Docker CE をインストールした場合

  * コンテナ内にインストールされたプログラミング言語やツールを使用するなら、Dev Containers 拡張機能を実行して開発コンテナ環境の VS Code に移行する必要がある。
  * コンテナ自体を操作、管理するなら、WSL 拡張機能を実行して WSL の Linux 環境の VS Code に移行する必要がある（Container Tools 拡張機能を使用できる）。
  * Windows 側のツール（Gemini CLI など）を使用するなら、Windows 環境の VS Code に移行する必要がある。

このように、3 つの環境を行き来しながら開発していくことになる。

### 開発コンテナ構成ファイル

**開発コンテナ構成ファイル**（development container configuration files）は、`.devcontainer` フォルダに配置された、以下のファイルの総称である。

  * `devcontainer.json` （必須の JSON ファイル）
  * Compose ファイル （任意）
  * Dockerfile （任意）

Dev Containers 拡張機能は、開発コンテナ構成ファイルを読み込んで、コンテナとコンテナ環境の VS Code のセットアップを自動化する。

読み込まれる開発コンテナ構成ファイルの優先順位は

  1. プロジェクトフォルダ（**ワークスペース**）直下の `.devcontainer` フォルダに配置されたもの
  2. 以下の**ユーザーデータフォルダ**にある `.devcontainer` フォルダに配置されたもの
      * Windows: `$Env:APPDATA\Code\User\globalStorage\ms-vscode-remote.remote-containers`
      * macOS: `~/Library/Application Support/Code/User/globalStorage/ms-vscode-remote.remote-containers`
      * Linux: `~/.config/Code/User/globalStorage/ms-vscode-remote.remote-containers`

となる。プロジェクトフォルダ配下の開発コンテナ構成ファイルをプロジェクトのバージョン管理（Git など）の対象に含めると、リポジトリをクローンした時点で開発環境の構成も手に入ることになって便利である。

Dev Containers 拡張機能は、開発コンテナ構成ファイルを自動生成するコマンドや、開発コンテナ構成ファイルに基づいてコンテナをビルドするコマンドなどを提供する。

Dev Containers 拡張機能のコマンドは

  * <kbd>Ctrl</kbd> + <kbd>Shift</kbd> + <kbd>P</kbd> キーを同時に押してコマンドパレットを開き、`devcontainer`を入力して表示される項目から実行できる。
  * または、VS Code 画面の左下にある「><」ボタンを押して開くコマンドパレットから実行できる。

Windows などのローカル環境の VS Code では、以下のコマンドを実行できる。

| コマンド | 機能 |
|:---|:---|
| Add Development Container Configuration Files...<br /><br />（開発コンテナー構成ファイルを追加...） | テンプレートを選んで開発コンテナ構成ファイルを自動生成する |
| Reopen in Container<br /><br />（コンテナーで再度開く） | コンテナを実行し、コンテナ環境の VS Code に移行する。また、ローカル環境で開いていたフォルダを、<br /><br />コンテナ内のファイルシステムにマウントする。初回ではコンテナをセットアップするため時間がかかる |
| Attach to Running Container...<br /><br />（実行中のコンテナーにアタッチ...） | 実行中コンテナ環境の VS Code に移行する |
| Rebuild and Reopen in Container<br /><br />（コンテナ―でリビルドして再度開く） | `Reopen in Container` とほとんど同じだが、コンテナを実行するまえにイメージを強制的に再構築す<br /><br />る。古いビルドキャッシュを使い回して Dockerfile の変更が反映されない場合に使う。このコマンドは<br /><br />「><」ボタンを押して開くコマンドパレットのメニューからはアクセスできない |

開発コンテナ環境の VS Code では、以下のコマンドを実行できる。

| コマンド | 機能 |
|:---|:---|
| Reopen Folder Locally<br /><br />（フォルダーをローカルで再度開く） | ローカル環境の VS Code に戻り、ローカル側のフォルダを開く。コンテナは停止する |
| Rebuild Container<br /><br />（コンテナーのリビルド） | 書き換えられた開発コンテナ構成ファイルの再読み込みをしてコンテナを作り直す |

■ 単独コンテナの場合

コンテナ構成テンプレートに Ubuntu 24.04 LTS（Noble Numbat）を選び、追加機能に Python のみを選んだ場合、次のような `.devcontainer\devcontainer.json` ファイルが自動生成される:

``` javascript
{
    "name": "Ubuntu",
    "image": "mcr.microsoft.com/devcontainers/base:noble",
    "features": {
        "ghcr.io/devcontainers/features/python:1": {}
    }
}
```

| プロパティ | データ型 | 値 |
|:---|:---|:---|
| `name` | 文字列 | エディタの UI に表示される開発コンテナの名前 |
| `image` | 文字列 | 開発コンテナの基となる Docker イメージをリポジトリ名とタグで指定する。よく使われるのは MCR（Microsoft Container Registry）で<br /><br />提供されている Microsoft 製の Docker イメージ |
| `features` | オブジェクト | 追加機能の設定。各追加機能ごとに、「機能 ID」プロパティとオプションを指定する |

追加機能の「機能 ID」とオプションは、次のサイトで探すことができる。

  * [Available Dev Container Features](https://containers.dev/features)

■ 複数コンテナの場合

複数コンテナを使用する場合、Compose ファイルが必要であり、Compose ファイルを参照する `devcontainer.json` を作成する。テンプレートの中には、複数コンテナに対応するものもある。

例えば、「Python 3 & PostgreSQL」テンプレートを選ぶと、次のような `devcontainer.json` が自動生成される:

``` javascript
{
    "name": "Python 3 & PostgreSQL",
    "dockerComposeFile": "docker-compose.yml",
    "service": "app",
    "workspaceFolder": "/workspaces/${localWorkspaceFolderBasename}"
}
```

| プロパティ | データ型 | 値 |
|:---|:---|:---|
| `dockerComposeFile` | 文字列<br /><br />or 配列 | Compose ファイルを指定する。単一の文字列（ファイルパス）または複数ファイルの配列を指定できる。環境依存を避けるため<br /><br />相対パスが基本で、基準は `devcontainer.json` のあるディレクトリ |
| `service` | 文字列 | Compose ファイルで定義されたサービスのうち、開発コンテナが作られるサービス。ローカルのプロジェクトフォルダはそのコ<br /><br />ンテナ内のディレクトリにバインドマウントされる（WSL2 の場合は自動マウントとバインドマウントの 2 段構成） |
| `workspaceFolder` | 文字列 | 開発コンテナ内のワークスペースフォルダのパス。これは通常、バインドマウントのターゲットパスである |
| `overrideCommand` | 真理値 | `true`（デフォルト）の場合、Dev Containers 拡張は、開発コンテナの起動時に `/bin/sh -c "while sleep 1000; do :; done"`<br /><br />（シェルを起動するだけの軽いコマンド）を実行するようにコンテナの設定（Dockerfile の `ENTRYPOINT`、Compose ファイルの<br /><br /> `command`）を上書きする。Dockerfile の `ENTRYPOINT`、Compose ファイルの `command` を実行したい場合は、`false` を設定す<br /><br />る必要がある |

テンプレートからは Compose ファイルと Dockerfile も自動生成される。

例えば、「Python 3 & PostgreSQL」テンプレートを選ぶと、次のような `docker-compose.yml` が生成される。

``` yaml
services:
  app:
    build:
      context: ..
      dockerfile: .devcontainer/Dockerfile
    volumes:
      - ../..:/workspaces:cached
    command: sleep infinity
    network_mode: service:db
  db:
    image: postgres:latest
    restart: unless-stopped
    volumes:
      - postgres-data:/var/lib/postgresql/data
    environment:
      POSTGRES_USER: postgres
      POSTGRES_DB: postgres
      POSTGRES_PASSWORD: postgres
volumes:
  postgres-data:
```

`app` サービスの定義において、`db` サービスとネットワーク名前空間を共有する `network_mode` の設定が含まれていることに注意する。

サービス名による名前解決を使いたい場合には、`network_mode` の行を削除またはコメントアウトする必要がある。

### 追加のマウントの設定

`devcontainer.json` で追加のマウントを設定できる。Docker CLI の `--mount` オプションを JSON で書けるイメージになる。

| プロパティ | データ型 | 値 |
|:---|:---|:---|
| `mounts` | 配列 | 開発コンテナの追加のボリュームやバインドマウントの設定。値は Docker CLI の `--mount` オプションと同じ書式で記述する文字列、<br /><br />またはオブジェクトの配列とする |

オブジェクトのキーは、以下のとおり:

| プロパティ | データ型 | 値 |
|:---|:---|:---|
| `type` | 文字列 | `bind`（バインドマウント）、`volume`（ボリュームマウント） |
| `source` | 文字列 | ホスト側のパスやボリューム名 |
| `target` | 文字列 | コンテナ内でマウントするパス |
| `consistency` | 文字列 | Linux 以外のプラットフォームでのファイル同期モード<br /><br />・`consistent`: 常にホストとコンテナが完全に同じ表示（デフォルト）<br /><br />・`delegated`: コンテナ上の更新がホスト上に反映するまで、遅延が発生するのを許容<br /><br />・`cached`: ホスト上の更新がコンテナ上に反映するまで、遅延が発生するのを許容<br /><br />多くのケースでは、コンテナ内に書き込んだファイルを、即時ホスト上に反映する必要がないので、`delegated` にすることでパフォー<br /><br />マンスを最適化できる |

`mounts` プロパティでは、環境依存のパスを柔軟に扱うための変数（プレースホルダ）展開が可能。代表的なものは以下のとおり:

| 変数 | 展開内容 |
|:---|:---|
| `${localWorkspaceFolder}` | ホスト側のワークスペースの絶対パス |
| `${containerWorkspaceFolder}` | コンテナ内でのワークスペースパス |
| `${localEnv:VAR_NAME}` | ホストの環境変数 VAR_NAME の値 |
| `${containerEnv:VAR_NAME}` | コンテナ内の環境変数 VAR_NAME の値 |
| `${devcontainerId}` | 開発コンテナごとに一意な ID |
| `${userHome}` | ホストのユーザーのホームディレクトリ |
| `${localWorkspaceFolderBasename}` | ワークスペースフォルダ名（パスを除いた末尾名） |

`mounts` の設定例:

``` javascript
{
    "mounts": [
        {
            "type": "bind",
            "source": "${localWorkspaceFolder}",
            "target": "${containerWorkspaceFolder}",
            "consistency": "delegated"
        }
    ]
}
```

### ポートフォワード機能の設定

開発コンテナ内で開いているポートは VS Code が検出して自動的にローカル環境へ転送されるため、通常は localhost からそのサービスにアクセスできる。

しかし、自動検出に頼るだけでなく、特定のポートが必ずローカル環境に転送されるように指定したい場合もある。とくに、開発コンテナ以外のコンテナ内で開いているポートは開発コンテナ環境の VS Code では検出されないため、そのままではローカルからアクセスできない。

例えば、Web アプリケーション開発のプロジェクトにおいて、Compose ファイルで `app` サービス、`db` サービス、`proxy` サービスを定義し、それぞれ、アプリケーション本体（Django）、データベースサーバー（PostgreSQL）、HTTP サーバー（nginx）が入るコンテナを実行するケースを考える。アプリケーション本体が入るコンテナを開発コンテナとするが、それを直接公開するのではなく、プロキシとして HTTP サーバーを置く構成とする。これにより HTTPS などの高度な機能を HTTP サーバーに依存することができる。

![](https://www.plantuml.com/plantuml/png/SoWkIImgAStDuKfCJyqhKNZSFErnyyh7JSiUDstZAaXCpWOAAoGM52XKS-NbfwGgk6GbvgNd0zabgGf9-IKb-RbM2Y-dTJ-VkUjnq-R7pK3KzhYvYwiMhZSb2QOr8B-u9BqeDJZG1qcwe2Y_eXAc8YyzCwy2A0cc5MnqTL6S0nNMjOCv0a811P2DTaZDIm5w1m00)

HTTP サーバーが入るコンテナは、開発コンテナとは別なので VS Code がポートを検出できない。このままでは、Web ブラウザが HTTP サーバーに localhost アクセスすることができない。

このような場合、自動検出に頼らず `devcontainer.json` ファイル内で明示的にポート転送を設定することができる。

| プロパティ | データ型 | 値 |
|:---|:---|:---|
| `forwardPorts` | 配列 | ローカル環境に常に転送されるポート番号または `host:port` 値の配列。`host` 部分には、Docker Compose のネットワーク<br /><br />内で解決可能なサービス名を指定できる |
| `portsAttributes` | オブジェクト | コンテナ内で開かれるポートごとに細かい挙動を制御するための設定 |

`portsAttributes` で利用可能なオプションは次のとおり:

| プロパティ | データ型 | 値 |
|:---|:---|:---|
| `label` | 文字列 | ポートにわかりやすい名前を付ける。VS Code の「PORTS」ビューに表示される |
| `onAutoForward` | 文字列 | 自動転送されたときの挙動を指定する<br /><br />・`notify`: 通知を出す（デフォルト）<br /><br />・`openBrowser`: ブラウザで自動的に開く<br /><br />・`openPreview`: VS Code 内のプレビューで開く<br /><br />・`silent`: 何も表示しない |
| `requireLocalPort` | 真理値 | `true` にすると、必ずローカル側の同じポート番号を使う。競合している場合はエラーになる |

`forwardPorts` と `portsAttributes` の設定例:

``` javascript
{
    "forwardPorts": [3000, "proxy:8080"]
    "portsAttributes": {
        "3000": {
            "label": "Web App",
            "onAutoForward": "openBrowser",
            "requireLocalPort": true
        }
    }
}
```

この例では、開発コンテナ内の 3000 番ポートと、`proxy` サービスが起動するコンテナ内の 8080 番ポートがローカル環境に転送され、localhost からアクセスできる。また、開発コンテナ内の 3000 番ポートは、必ずローカル側の同じポート番号を使い、ブラウザが起動して自動的に開く。

なお、Windows の場合、WSL2 の自動ポートフォワード機能と VS Code の自動ポートフォワード機能が 2 重になって機能していることになる。WSL2 の自動転送は WSL2 の Linux 内のポートが転送されるのに対して、VS Code の自動転送は開発コンテナ内で開いているポートが転送される。下図で整理する。

![](https://www.plantuml.com/plantuml/png/SoWkIImgAStDuKfCJyqhKL0AJqrIUDmyxN7poiTDonutRLE2IsJcfUIN5mCbPpzkslcukGCk4nQiu9oVagh4bPnOb09K6EnZfE2JcLTQeQHG3LNpk6dHu-RfZvjNSQKL9UQcAYWzRbxsk7aFq7mW-NJThEVxLYea5MMcfAS2pUzztDmCDFznytgmalKWO_BoKu2wNaxhVZvpr-EcpO-RWQZjSNEN4aGc9Och5WWnNQ-WQuVDwGfM2ZdvoOav6Vd59LoOyc1j27_1M11XJ4U0Z7lHlkffzzGNknUzR6Zamg86h3Sb2QZowOJUfvszBvOlVhPn5mwx4U65RZA90QSRdP6PafAQ5wkB50KEgNafm1W70000)

### 環境変数などの設定

`devcontainer.json` で開発コンテナ内での環境変数、ユーザー、コマンド実行を設定できる。

| プロパティ | データ型 | 値 |
|:---|:---|:---|
| `remoteEnv` | オブジェクト | 開発コンテナ接続時のみに反映される環境変数。キーを環境変数とする。値を `${localEnv:<変数>}` とするとホストの<br /><br />環境変数の値を引き継ぐ。このように値が静的でない場合は、コンテナ全体を再ビルドしなくても値を更新できる |
| `remoteUser` | 文字列 | 開発コンテナ内で接続するユーザー。デフォルトでは、コンテナ全体が実行されているユーザー |
| `postCreateCommand` | 文字列<br /><br />or 配列<br /><br />or オブジェクト |開発コンテナ作成時の 1 回のみ実行されるコマンドを文字列で指定する |

``` javascript
{
    "remoteEnv": {
        "API_URL": "https://api.dev.local",
        "LOCAL_PATH": "${localEnv:HOME}"
    },
    "remoteUser": "root",
    "postCreateCommand": "pip install -r requirements.txt"
}
```

### IDE の設定

`devcontainer.json` のトップレベルの `customizations` プロパティは、開発コンテナ環境を利用するエディタやツールごとに追加の設定を行うためのセクションである。

開発コンテナ環境の VS Code は拡張機能と設定を独自に管理しており、とくに拡張機能については基本的にローカル環境の VS Code から引き継がない（拡張機能の中には、テンプレートにより自動的に開発コンテナ環境の VS Code にインストールされるものもある）。

`customization` プロパティに設定するオブジェクトのプロパティとして `vscode` キーを使うことで、開発コンテナ環境の VS Code の拡張機能と設定を自動適用できる。

| プロパティ | データ型 | 値 |
|:---|:---|:---|
| `extensions` | 配列 | 開発コンテナ作成時に開発コンテナ内にインストールする必要がある拡張機能を指定する拡張機能 ID の配列。デフォルトは `[]` |
| `settings` | オブジェクト | ユーザー設定を上書きする。デフォルトは `{}` |

`vscode` キーの設定例:

``` javascript
{
    "customizations": {
        "vscode": {
            "settings": {
                "editor.tabSize": 2,
                "python.analysis.typeCheckingMode": "strict"
            },
            "extensions": [
                "ms-python.python",
                "charliermarsh.ruff"
            ]
        }
    }
}
```

`extensions` プロパティのほうは、手作業で編集する以外の方法がある。

アクティビティーバーから「拡張機能」サイドバーを開き、その「インストール済み」欄にある拡張機能の ⚙️ アイコンから「devcontainer.json に追加」を選択することにより、`extensions` プロパティが書き加えられる。

全ての開発コンテナに共通でインストール自動化したい拡張機能は、VS Code の設定ファイル `settings.json` の `"dev.containers.defaultExtensions"` プロパティで設定できる。<kbd>Ctrl + Shift + P</kbd>（Windows / Linux）または <kbd>Cmd + Shift + P</kbd>（macOS）でコマンドパレットを開き、`settings` を入力、「Preferences: Open User Settings (JSON)」（基本設定: ユーザー設定を開く (JSON)） を選択すると、`settings.json` を開くことができる。

`settings.json` 設定例:

``` javascript
{
    // 他の記述...
    "dev.containers.defaultExtensions": [
        "naumovs.color-highlight",
        "github.copilot",
        "github.copilot-chat",
        "streetsidesoftware.code-spell-checker",
    ],
}
```